# Configuration

Project configuration is stored in `jurassic.json` file

In [1]:
//| export

import { z } from "zod";
import path from "node:path";

In [2]:
//| export

const configSchema: z.Schema = z.object({
  configPath: z.string(),
  nbsPath: z.string().default("."),
  outputPath: z.string().default("."),
  docsPath: z.string().default("."),
});

export type Config = z.infer<typeof configSchema>;

🕵️‍♀️ Looking for config. Start from `cwd` and keep going up if needed looking
for `jurassic.json`. When running notebooks, it seems like `cwd` points to
notebook's directory (at least when running in VS Code). Hence this extra
gymnastics, just to be on the safe side. Notice `d` (depth) and `maxD` (max
depth) to make sure things don't get out of control

In [3]:
//| export

const findConfig = async (
  dir: string = Deno.cwd(),
  d = 0,
  config = "jurassic.json",
  maxD = 10,
): Promise<string> => {
  if (d >= maxD) throw new Error("max depth reached");

  try {
    const f = path.join(dir, config);
    await Deno.lstat(f);
    return f;
  } catch {
    return findConfig(path.join(dir, "../"), d + 1);
  }
};

In [4]:
await findConfig();

"/Users/philip/projects/jurassic/jurassic.json"

In [5]:
//| export

export const getTestConfig = (baseDir: string): Config =>
  configSchema.parse({
    configPath: path.resolve(baseDir, "jurassic.json"),
    nbsPath: path.resolve(baseDir, "nbs"),
    outputPath: path.resolve(baseDir, "jurassic"),
    docsPath: path.resolve(baseDir, "docs"),
  });

In [6]:
getTestConfig("./");

{
  configPath: "/Users/philip/projects/jurassic/nbs/jurassic.json",
  nbsPath: "/Users/philip/projects/jurassic/nbs/nbs",
  outputPath: "/Users/philip/projects/jurassic/nbs/jurassic",
  docsPath: "/Users/philip/projects/jurassic/nbs/docs"
}

Load and parse config.

In [7]:
//| export

export const getConfig = async (): Promise<Config> => {
  const cf = await findConfig();
  const dcf = path.dirname(cf);
  const c = configSchema.parse(
    Object.assign({ configPath: cf }, JSON.parse(await Deno.readTextFile(cf))),
  );
  c.nbsPath = path.join(dcf, c.nbsPath);
  c.outputPath = path.join(dcf, c.outputPath);
  c.docsPath = path.join(dcf, c.docsPath);
  return c;
};